In [1049]:
import os, sys, json, webbrowser, pprint
import spotipy 
import lyricsgenius
import pandas as pd
import nltk
import spotipy.util as util
import spotipy.oauth2 as oauth2
from json.decoder import JSONDecodeError
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from requests.exceptions import Timeout


In [1050]:
# Define variables
username = 'g64129687775'
client_id = 'd576e9eb16044adbaa2d22688fc73dd0'
client_secret = '7b5cc4d0a7ce40ee9f8c0ea42aba241b'
redirect_uri = 'http://localhost/'
scope='user-read-recently-played'

In [1051]:
# Create tokens
token = SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope,username=username,show_dialog=True)
genius_access_token = 'cQ15SgX_iCubNYTUW2pivTNrZbHmUIBHlzUHg2GEbDbrFUmzEVaRFE9BPvefe2io'

In [1052]:
# Create API objects
spotifyObject = spotipy.Spotify(auth_manager=token)
geniusObject = lyricsgenius.Genius(genius_access_token)

In [1053]:
# Create NLP analyser
analyser = SentimentIntensityAnalyzer()

In [1054]:
# Get playlist
playlist = spotifyObject.playlist('spotify:playlist:37i9dQZF1DX2yvmlOdMYzV')
mood = 8
playlist_name = playlist['name']

In [1055]:
results = playlist['tracks']
p = results['items']
while results['next']:
    results = spotifyObject.next(results)
    p.extend(results['items'])

In [1056]:
if mood == 1:
    mood = 'Happy'
elif mood == 2:
    mood = 'Sad'
elif mood == 3:
    mood = 'Calm'
elif mood == 4:
    mood = 'Sleepy'
elif mood == 5:
    mood = 'Energised'
elif mood == 6:
    mood = 'Aroused'
elif mood == 7:
    mood = 'Angry'
elif mood == 8:
    mood = 'Chill'

In [1057]:

# p_item = playlist['tracks']['items']

# track_id = p_item[0]['track']['id']
# track_name = p_item[0]['track']['name']
# track_artists = p_item[0]['track']['artists']
# song = geniusObject.search_song(title=track_name, artist=track_artists[0]['name'])
# annotation = geniusObject.song_annotations(song.id)
# print(song.lyrics+'\n')
# print(annotation)



In [1058]:
geniusObject.timeout = 15

In [1059]:
songs = []
for x in p:
    track_id = x['track']['id']
    track_name = x['track']['name']
    track_artists = x['track']['artists']
    
    annotation = None
    lyric_score = None
    annotation_score = None
    
    features = spotifyObject.audio_features(track_id)[0]

    retries = 0
    while retries < 3:
        try:
            song = geniusObject.search_song(title=track_name, artist=track_artists[0]['name'])
        except Timeout as e:
            retries += 1
            continue            
        break


    try:
        annotation = geniusObject.song_annotations(song.id)
        lyric_score = analyser.polarity_scores(song.to_text())
        annotation_score = analyser.polarity_scores(song.to_text())
    except Exception as e:
        print('############################')
        print(e)
        print('Error 1 For', track_name)
        print('############################')

    try:
        songs.append({
        # Spotify features
        'name' : track_name,
        'artists' : [d['name'] for d in track_artists],
        'acousticness' : features['acousticness'],
        'danceability' : features['danceability'],
        'duration_ms' : features['duration_ms'],
        'energy' : features['energy'],
        'instrumentalness' : features['instrumentalness'],
        'key' : features['key'],
        'liveness' : features['liveness'],
        'loudness' : features['loudness'],
        'mode' : features['mode'],
        'speechiness' : features['speechiness'],
        'tempo' : features['tempo'],
        'time_signature' : features['time_signature'],
        'valence' : features['valence'],
        
        # Genius features
        # 'lyrics' : song.lyrics if song else None,
        # 'annotations' : annotation if annotation else None,
        'nlp_lyrics' : lyric_score['compound'] if lyric_score else None,
        'nlp_annotations' : annotation_score['compound'] if annotation_score else None,
        
        'valence+nlp' : round(features['valence']+(lyric_score['compound']+annotation_score['compound'])/100, 4) if song else features['valence'],
        'mood': mood 
        })
    except Exception as e:
        print('############################')
        print(e)
        print('Error 2 For', track_name)
        print('############################')


Searching for "North Face" by ODIE...
Done.
Searching for "Remember Me" by UMI...
Done.
Searching for "Mine" by Alex Isley...
Done.
Searching for "Shine" by Cleo Sol...
Done.
Searching for "Loverboy" by Joesef...


KeyboardInterrupt: 

In [1047]:
df = pd.DataFrame(songs)
df

,name,artists,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,nlp_lyrics,nlp_annotations,valence+nlp,mood
0,Remember Me,[UMI],0.45600,0.840,199227,0.344,0.000034,5,0.3500,-8.613,0,0.0374,111.994,4,0.526,0.7286,0.7286,0.5406,Chill
1,North Face,[ODIE],0.79200,0.802,196800,0.382,0.163000,10,0.0783,-7.356,1,0.0312,99.969,4,0.581,0.9486,0.9486,0.6000,Chill
2,Mine,"[Alex Isley, Jack Dine]",0.82800,0.347,212571,0.395,0.000011,6,0.1250,-9.278,0,0.0567,67.492,4,0.133,0.8880,0.8880,0.1508,Chill
3,Shine,[Cleo Sol],0.68600,0.742,226118,0.504,0.517000,1,0.1030,-10.105,1,0.0392,140.000,4,0.601,0.9515,0.9515,0.6200,Chill
4,Loverboy,[Joesef],0.59400,0.356,238123,0.611,0.000000,11,0.1190,-7.219,0,0.0567,79.338,4,0.524,0.9765,0.9765,0.5435,Chill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Blind Man,[Xavier Omär],0.45300,0.890,242196,0.523,0.000002,6,0.0831,-8.526,1,0.0560,111.031,4,0.345,0.9992,0.9992,0.3650,Chill
96,Peaches (feat. Daniel Caesar & Giveon),"[Justin Bieber, Daniel Caesar, Giveon]",0.32100,0.677,198082,0.696,0.000000,0,0.4200,-6.181,1,0.1190,90.030,4,0.464,0.9889,0.9889,0.4838,Chill
97,So Good At Being in Trouble,[Unknown Mortal Orchestra],0.03630,0.829,230147,0.435,0.878000,0,0.1190,-10.136,1,0.0515,103.816,4,0.594,-0.7326,-0.7326,0.5793,Chill
98,Somehow.,[Phony Ppl],0.71900,0.502,230973,0.399,0.003630,11,0.6450,-9.934,1,0.0295,92.904,4,0.124,0.2887,0.2887,0.1298,Chill


In [1048]:
invalid = '<>:"/\|?*'

for char in invalid:
	playlist_name = playlist_name.replace(char, '')

df.to_csv('datasets/' + playlist_name + '.csv')